# Data Science - Project Final

Danh sách thành viên:

|MSSV|Họ và Tên
|:---|:-
|18120296|Cao Tất Cường
|18120339|Hà Văn Duy



## Tổng quan

**COVID-19**

Đây là đại dịch bệnh tryền nhiễm nguy hiểm, phát hiện lần đầu tiên ở Trung Quốc sau đó nhanh chóng lan rộng ra toàn thế giới.  
Người mắc phải virus gây bệnh (SARS-CoV-2) thường sẽ có một khoảng thời gian ủ bệnh từ **2 - 14 ngày**, sau đó xuất hiện triệu chứng, một vài trường hợp có thể diễn tiến rất nhanh trở thành ca bệnh nặng, có thể dẫn đến tử vong.

Thời điểm hiện tại, vacxin vẫn chưa được phân phối rộng rãi và kiểm định kết quả, đồng thời vẫn chưa có thuốc chữa đặc hiệu. Vì vậy, các bệnh nhân cần được chăm sóc y tế hồi sức để cơ thể tự sản sinh ra đề kháng và khỏi bệnh.  
Đây là một dịch bệnh nghiêm trọng, vì vậy việc theo dõi sự tác động của nó với xã hội là điều cực kì cần thiết nhằm đưa ra các chính sách, giải pháp phù hợp để cải thiện tình hình, tiến tới xóa bỏ đại dịch trong tương lai gần.



https://vietnamese.cdc.gov/coronavirus/2019-ncov/cases-updates/about-us-cases-deaths.html


## Cài đặt và import các thư viện cần thiết

In [1]:
# handle output to /dev/null to ignore text outputs xD
!pip install sodapy > /dev/null

# update sklearn to support set_config attr (see this answer: https://stackoverflow.com/a/62481510)
!pip install --upgrade scikit-learn > /dev/null

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn import set_config
set_config(display='diagram')

from sodapy import Socrata

import regex as re
from sklearn.metrics import r2_score

## Thu thập dữ liệu

In [3]:
client = Socrata("data.cdc.gov", None)
 
results = client.get("9mfq-cb36", limit=30000)
 
data_df = pd.json_normalize(results)

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [4]:
data_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-05-04T00:00:00.000,KY,5245,5245.0,0,115.0,0,261,261.0,0,8.0,0,2020-05-05T17:25:08.000,Agree,Agree
1,2020-12-08T00:00:00.000,OK,205999,171497.0,34502,0.0,0,1752,1680.0,72,0.0,0,2020-12-09T14:45:40.234,Agree,Agree
2,2020-04-30T00:00:00.000,IA,7145,NaN,NaN,302.0,0,162,NaN,NaN,14.0,0,2020-05-01T21:00:19.025,Not agree,Not agree
3,2020-06-25T00:00:00.000,NE,18346,NaN,NaN,125.0,0,260,NaN,NaN,3.0,0,2020-06-26T19:18:27.809,Not agree,Not agree
4,2020-02-24T00:00:00.000,CA,10,NaN,NaN,0.0,NaN,0,NaN,NaN,0.0,NaN,2020-03-26T16:22:39.452,Not agree,Not agree


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [5]:
data_df.shape

(20520, 15)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát sơ bộ dữ liệu ta thấy mỗi dòng chứa các thông tin về số tổng ca nhiễm, số ca nhiễm mới, sô ca tử vong,... của của 1 bang trong một ngày cụ thể.

### Dữ liệu có các dòng bị lặp không?

In [6]:
data_df.index.duplicated().sum()

0

### Mỗi cột có ý nghĩa gì?

|VARIABLE|DESCRIPTION|TYPE|
|---|----------------|------|
|submission_date|Date of counts|Date & Time|      
|state|Jurisdiction|Plain Text|  
|tot_cases|Total number of cases|Number| 
|conf_cases|Total confirmed cases|Number|
|prob_cases|Total probable cases|Number|
|new_case|Number of new cases|Number|
|pnew_case|Number of new probable cases|Number|
|tot_death|Total number of deaths|Number|
|conf_death|Total number of confirmed deaths|Number|
|prob_death|Total number of probable deaths|Number|
|new_death|Number of new deaths|Number|
|pnew_death|Number of new probable deaths|Number|
|created_at|Date and time record was created|Date & Time|
|consent_cases|If Agree, then confirmed and probable cases are included. If Not Agree, then only total cases are included|Plain Text|
|consent_deaths|If Agree, then confirmed and probable deaths are included. If Not Agree, then only total deaths are included|Plain Text|


	


	


	


	




## Đưa ra câu hỏi cần trả lời

Số ca mắc mới trong một ngày bất kì (hợp lệ) là bao nhiêu?

## Tiền xử lý dữ liệu để phù hợp với bài toán

In [7]:
# number of records will be collected
callback_days = 28

def add_row_df(df):
    df = df.sort_values('submission_date', ascending=True)

    # Validation data
    for g in df.groupby('submission_date'):
        if g[1].shape[0] > 1:
            raise Exception("Error, please groupby dataset by date before pass to this func")
    
    # MAGIC
    new_case_data = df['new_case']
    
    l = None

    for j in range(len(new_case_data)):
        if j >= callback_days:
            t = new_case_data.iloc[j - callback_days: j]
            
            # append row
            l = np.r_[l, [np.flip(t)]]
        else:
            t = new_case_data.iloc[0: j + 1]
            fill = np.r_[np.array(['0' for i in range(callback_days - len(t))]), t]

            # append row
            if l is None:
                l = np.array([np.flip(fill)])
            else:
                l = np.r_[l, [np.flip(fill)]]
    
    df2 = pd.DataFrame(l, index=df.index, columns=[f"new_case_his_{i}" for i in range(1, callback_days + 1)])
    
    return pd.concat([df, df2], axis=1)

In [8]:
collected_df = data_df.groupby('state').apply(add_row_df).reset_index(level='state', drop=True)

In [9]:
# correct? test this cell
#collected_df.groupby('state').get_group('AK').sort_values('submission_date', ascending=True).tail(10) 

Ép về đúng kiểu dữ liệu

In [10]:
whitelist = ['submission_date', 'state', 'created_at','consent_cases','consent_deaths']

collected_df = collected_df.transform(lambda x: x if (x.name in whitelist) else pd.to_numeric(x, errors='coerce'))

Loại bỏ các dòng dữ liệu chắc chắn là nhiễu (những ngày chưa ghi nhận ca mắc nào)

In [11]:
#add_row_df(results_df)
collected_df = collected_df[collected_df['tot_cases'] > 0]
collected_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,...,new_case_his_19,new_case_his_20,new_case_his_21,new_case_his_22,new_case_his_23,new_case_his_24,new_case_his_25,new_case_his_26,new_case_his_27,new_case_his_28
18512,2020-03-13T00:00:00.000,AK,1,NaN,NaN,1.0,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9152,2020-03-14T00:00:00.000,AK,1,NaN,NaN,0.0,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19051,2020-03-15T00:00:00.000,AK,1,NaN,NaN,0.0,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17452,2020-03-16T00:00:00.000,AK,1,NaN,NaN,0.0,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7738,2020-03-17T00:00:00.000,AK,3,NaN,NaN,2.0,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Đến đây ta đã thu thập đủ dữ liệu được lưu trong `collected_df` để có thể đặt câu hỏi.

## Đặt câu hỏi: Số ca mắc mới trong một ngày bất kì (hợp lệ) là bao nhiêu?

## Tiền xử lý (tách tập)

In [12]:
sorted_df = collected_df.sort_values('submission_date', ascending=True)
X_df = sorted_df.drop('new_case', axis=1)
y_sr = sorted_df['new_case']

Tách theo tỉ lệ: 70% 15% 15% lần lượt cho bộ train, bộ validation, bộ test.

In [13]:
# tách bộ train và bộ (validation, test)

# bộ train sẽ dùng dữ liệu của 60 bang trong 300 ngày đầu tiên. 
# train_size = 60 * 300 

train_X_df, new_X_df, train_y_sr, new_y_sr = train_test_split(X_df, y_sr, train_size = 0.7, 
                                                             random_state=0)

Tách bộ validation và bộ test

In [14]:
val_X_df, test_X_df, val_y_sr, test_y_sr = train_test_split(new_X_df, new_y_sr, test_size=0.5, 
                                                              random_state=0)

In [15]:
train_X_df.shape

(11820, 42)

## Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [16]:
train_X_df.dtypes

submission_date     object
state               object
tot_cases            int64
conf_cases         float64
prob_cases         float64
pnew_case          float64
tot_death            int64
conf_death         float64
prob_death         float64
new_death          float64
pnew_death         float64
created_at          object
consent_cases       object
consent_deaths      object
new_case_his_1     float64
new_case_his_2     float64
new_case_his_3     float64
new_case_his_4     float64
new_case_his_5     float64
new_case_his_6     float64
new_case_his_7     float64
new_case_his_8     float64
new_case_his_9     float64
new_case_his_10    float64
new_case_his_11    float64
new_case_his_12    float64
new_case_his_13    float64
new_case_his_14    float64
new_case_his_15    float64
new_case_his_16    float64
new_case_his_17    float64
new_case_his_18    float64
new_case_his_19    float64
new_case_his_20    float64
new_case_his_21    float64
new_case_his_22    float64
new_case_his_23    float64
n

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [17]:
train_X_df.dtypes[train_X_df.dtypes != object]

tot_cases            int64
conf_cases         float64
prob_cases         float64
pnew_case          float64
tot_death            int64
conf_death         float64
prob_death         float64
new_death          float64
pnew_death         float64
new_case_his_1     float64
new_case_his_2     float64
new_case_his_3     float64
new_case_his_4     float64
new_case_his_5     float64
new_case_his_6     float64
new_case_his_7     float64
new_case_his_8     float64
new_case_his_9     float64
new_case_his_10    float64
new_case_his_11    float64
new_case_his_12    float64
new_case_his_13    float64
new_case_his_14    float64
new_case_his_15    float64
new_case_his_16    float64
new_case_his_17    float64
new_case_his_18    float64
new_case_his_19    float64
new_case_his_20    float64
new_case_his_21    float64
new_case_his_22    float64
new_case_his_23    float64
new_case_his_24    float64
new_case_his_25    float64
new_case_his_26    float64
new_case_his_27    float64
new_case_his_28    float64
d

In [18]:
num_cols = list(set(train_X_df.columns) - set(whitelist))
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,new_case_his_15,new_case_his_16,new_case_his_5,new_case_his_23,new_case_his_8,new_case_his_11,new_case_his_12,new_case_his_3,new_case_his_22,new_case_his_19,...,new_case_his_4,pnew_death,new_case_his_1,new_case_his_7,new_case_his_10,new_case_his_18,pnew_case,new_case_his_2,new_case_his_14,new_case_his_20
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.8,0.0,0.0,0.0,0.0,14.2,0.0,0.0,0.0
min,-33355.0,-30134.0,-33355.0,-10427.0,-33355.0,-33355.0,-33355.0,-30134.0,-33355.0,-33355.0,...,-33355.0,-675.0,-33355.0,-33355.0,-33355.0,-10427.0,-6259.0,-33355.0,-33355.0,-30134.0
lower_quartile,28.0,26.0,45.0,17.0,38.0,35.0,33.0,48.0,18.0,23.0,...,45.0,0.0,52.0,40.0,35.0,23.0,0.0,49.0,29.0,21.0
median,306.5,297.0,352.5,268.0,333.0,323.0,319.0,360.0,273.0,292.5,...,355.0,0.0,370.0,337.5,328.0,294.0,0.0,360.5,307.0,284.0
upper_quartile,989.2,972.0,1099.0,924.0,1075.2,1032.0,1022.0,1116.0,931.2,961.0,...,1113.2,0.0,1144.2,1083.0,1041.2,967.0,38.0,1134.2,1000.2,947.2
max,71734.0,71734.0,71734.0,71734.0,71734.0,71734.0,53711.0,71734.0,49458.0,30851.0,...,71734.0,5482.0,53711.0,71734.0,53711.0,49458.0,67475.0,71734.0,71734.0,71734.0


### Với mỗi cột input có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [19]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

,created_at,submission_date,consent_deaths,consent_cases,state
missing_ratio,0.0,0.0,5.6,5.6,0.0
num_values,470,342,3,3,58
value_ratios,"{'2020-03-26T16:22:39.452': 7.4, '2020-05-25T15:38:40.000': 1.6, '2020-11-13T19:10:12.893': 1.2, '2020-04-03T16:22:39.452': 0.6, '2020-03-28T16:22:39.452': 0.6, '2020-04-07T16:22:39.452': 0.6, '20...","{'2020-11-10T00:00:00.000': 0.4, '2020-11-06T00:00:00.000': 0.4, '2020-12-11T00:00:00.000': 0.4, '2020-11-14T00:00:00.000': 0.4, '2020-10-11T00:00:00.000': 0.4, '2020-04-30T00:00:00.000': 0.4, '20...","{'Agree': 62.3, 'Not agree': 30.0, 'N/A': 7.7}","{'Agree': 58.3, 'Not agree': 30.3, 'N/A': 11.4}","{'AZ': 2.1, 'WA': 2.1, 'CA': 2.1, 'IL': 2.0, 'MA': 2.0, 'NJ': 1.9, 'NE': 1.9, 'NV': 1.9, 'VA': 1.9, 'NYC': 1.9, 'MD': 1.8, 'ND': 1.8, 'HI': 1.8, 'FL': 1.8, 'GA': 1.8, 'MO': 1.8, 'IN': 1.8, 'WI': 1..."


## Tiền xử lý tập huấn luyện

- Loại bỏ các thuộc tính dự đoán như `pnew_case`, `pnew_death`
- Loại bỏ các thuộc tính không có ích cho việc huấn luyện mô hình như `submission_date`, `state`, `conf_death`, `prob_death`, `created_at`, `consent_cases`, `consent_deaths`

- Dùng tham số `callback_days` để quy định chỉ lấy số ngày gần nhất của ngày hiện tại.




In [20]:
class ColDropper(BaseEstimator, TransformerMixin):
    def __init__(self, callback_days=14):
        self.callback_days = callback_days

    def fit(self, X_df, y=None):
        return self
    
    def mix_data(self, row):
        tot_cases = row.loc['tot_cases']
        if row.isnull().loc['conf_cases']:
            row.loc['conf_cases'] = tot_cases            

        return row

    def transform(self, X_df, y=None):

        # drop unsue attr
        df = X_df.drop(['pnew_case', 'pnew_death', 'submission_date', 'state', 'conf_death', 'prob_death', 'created_at',
                        'consent_cases', 'consent_deaths'], axis=1)

        # regex to match first number in any text
        matcher = re.compile(r'.*?(\d+)')
        
        columns_sr = df.columns.str.extract(r'(.*?\d+)', expand=False).dropna().to_series()
        list_drop_his = columns_sr.transform(lambda x: x if int(matcher.findall(x)[0]) > self.callback_days else np.nan).dropna().values.tolist()
        if len(list_drop_his) > 0:
            df = df.drop(list_drop_his, axis=1)

        return df.transform(self.mix_data, axis=1)


In [21]:
# TEST
elimator = ColDropper(callback_days=14)
elimator.fit(X_df)

ColDropper()

In [22]:
# elimator.transform(train_X_df).duplicated(keep='last').sum()
# 262 (if callback_days = 20) How?

In [23]:

tmp = elimator.transform(train_X_df)
tmp

,tot_cases,conf_cases,prob_cases,tot_death,new_death,new_case_his_1,new_case_his_2,new_case_his_3,new_case_his_4,new_case_his_5,new_case_his_6,new_case_his_7,new_case_his_8,new_case_his_9,new_case_his_10,new_case_his_11,new_case_his_12,new_case_his_13,new_case_his_14
7410,24.0,24.0,NaN,0.0,0.0,5.0,4.0,1.0,0.0,7.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
9764,7116.0,7116.0,0.0,158.0,0.0,132.0,233.0,250.0,357.0,410.0,276.0,213.0,215.0,284.0,297.0,211.0,500.0,247.0,163.0
13639,33439.0,31191.0,2248.0,2044.0,42.0,529.0,566.0,503.0,614.0,627.0,731.0,484.0,680.0,349.0,449.0,520.0,597.0,636.0,471.0
17306,57504.0,57504.0,0.0,630.0,16.0,701.0,1138.0,1249.0,1475.0,1236.0,1184.0,1371.0,869.0,1272.0,1642.0,1213.0,961.0,919.0,1098.0
20180,5215.0,4677.0,538.0,140.0,0.0,25.0,40.0,27.0,44.0,30.0,43.0,21.0,23.0,55.0,0.0,29.0,42.0,31.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20160,4197.0,3767.0,430.0,127.0,0.0,24.0,29.0,26.0,20.0,19.0,-1.0,9.0,16.0,11.0,18.0,5.0,17.0,5.0,12.0
4950,58694.0,51736.0,6958.0,572.0,10.0,1073.0,950.0,987.0,873.0,698.0,510.0,878.0,1094.0,718.0,645.0,584.0,597.0,365.0,613.0
13760,126868.0,117450.0,9418.0,9008.0,21.0,315.0,349.0,571.0,0.0,109.0,431.0,313.0,352.0,221.0,249.0,394.0,-882.0,1826.0,319.0
796,74506.0,74506.0,NaN,1218.0,1.0,627.0,795.0,1010.0,400.0,462.0,393.0,763.0,991.0,1105.0,885.0,818.0,625.0,615.0,1009.0


### Xây dựng pipeline



Các thuộc tính sẽ được fill bằng `mean` nhằm đảm bảo không còn nơi nào bị thiếu dữ liệu kể cả trên bộ test mới.

## Tiền xử lý + mô hình hóa

### Tìm mô hình tốt nhất

In [24]:
pipe_line = make_pipeline(ColDropper(), SimpleImputer(strategy='mean'),
            StandardScaler(),
            MLPRegressor(hidden_layer_sizes=(30), activation='tanh', solver='lbfgs', max_iter=5000, random_state=0))


pipe_line.fit(train_X_df, train_y_sr)

/home/codespace/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Pipeline(steps=[('coldropper', ColDropper()),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', hidden_layer_sizes=30,
                              max_iter=5000, random_state=0, solver='lbfgs'))])

Thử validation xem sao

In [25]:
train_err = (1 - r2_score(train_y_sr, pipe_line.predict(train_X_df))) * 100
train_err

28.055196603750776

In [26]:
val_err = (1 - r2_score(val_y_sr, pipe_line.predict(val_X_df))) * 100        
val_err

32.482707942425534

### Đánh giá mô hình tìm được